In [1]:
import xlwings as xw
import pandas as pd
import pyautocad

Импорт таблицы в Autocad

In [43]:
sheet = xw.sheets.active
df = sheet.range('A1').expand().options(pd.DataFrame, header=1, index=False).value
df['Номер'] = df['Номер'].apply(lambda x: str(int(float(x))))
df['Высота'] = df['Высота'].apply(lambda x: str(x))
df['Толщина'] = df['Толщина'].apply(lambda x: str(x))
df = df[['Номер', 'Наименование', 'Количество', 'Высота', 'Толщина', 'Состояние', 'Действие']]
df

,Номер,Наименование,Количество,Высота,Толщина,Состояние,Действие
0,1,Берёза пушистая,1,0.4,0.01,Хорошее,Удаление
1,2,Берёза пушистая,1,0.4,0.01,Хорошее,Удаление
2,3,Берёза пушистая,1,0.4,0.01,Хорошее,Удаление
3,4,Берёза пушистая,1,0.4,0.01,Хорошее,Удаление
4,5,береза,6 стволов,"4,4,4,4,4,4","5,5,5,5,5,5",Хорошее,Удаление
5,6,Ива ломкая,54 ствола,"5х14, 4х30, 3х8,2,1","7х3,4х11, 2х40",Хорошее,Удаление
6,7,береза,1,2.5,0.05,Хорошее,Удаление
7,8,Ольха серая,1,5.0,5.0,Хорошее,Удаление
8,9,Ольха серая,1,5.0,4.0,Хорошее,Удаление
9,10,береза,41.5 м.кв.,1.5,0.05,Хорошее,Удаление


In [55]:
from src.parsing import Splitter

def transform_unit(sizes: str | float, m2cm: bool = False, cm2m: bool = False) -> str:
    if sizes == '-':
        return sizes
    if m2cm:
        k = 100
    elif cm2m:
        k = 0.01
    else:
        raise ValueError("m2cm или cm2m должен быть True")
    sizes_cm = [str(size*k) for size in Splitter.size(str(sizes))]
    sizes = ','.join(sizes_cm) if len(sizes_cm) > 1 else sizes_cm[0]
    return compress_size(sizes)

def compress_size(sizes: str) -> str:
    if sizes == '-':
        return sizes
    sizes = Splitter.size(str(sizes))
    compressed_sizes = []
    i = 0
    while i < len(sizes):
        count = 1
        while i + 1 < len(sizes) and sizes[i] == sizes[i + 1]:
            count += 1
            i += 1
        value = int(float(sizes[i])) if float(sizes[i]) % 1 == 0 else float(sizes[i])
        if count > 1:
            compressed_sizes.append(f"{value}x{count}")
        else:
            compressed_sizes.append(str(value))
        i += 1
    print(compressed_sizes)
    return ','.join(compressed_sizes) if len(compressed_sizes) > 1 else compressed_sizes[0]

# TODO: отдельные кнопки для преобразования в см и в метры. и отдельная кнопка для сжатия описания размеров
df['Толщина'] = df['Толщина'].apply(lambda x: transform_unit(x, m2cm=True))

df

['10000']
['10000']
['10000']
['10000']
['5000000x6']
['7000000x3', '4000000x11', '2000000x40']
['50000']
['5000000']
['4000000']
['50000']
['60000']
['70000.00000000001']
['30000x3']
['10000']
['10000']
['10000']
['30000x3']
['120000']


,Номер,Наименование,Количество,Высота,Толщина,Состояние,Действие
0,1,Берёза пушистая,1,0.4,10000,Хорошее,Удаление
1,2,Берёза пушистая,1,0.4,10000,Хорошее,Удаление
2,3,Берёза пушистая,1,0.4,10000,Хорошее,Удаление
3,4,Берёза пушистая,1,0.4,10000,Хорошее,Удаление
4,5,береза,6 стволов,"4,4,4,4,4,4",5000000x6,Хорошее,Удаление
5,6,Ива ломкая,54 ствола,"5х14, 4х30, 3х8,2,1","7000000x3,4000000x11,2000000x40",Хорошее,Удаление
6,7,береза,1,2.5,50000,Хорошее,Удаление
7,8,Ольха серая,1,5.0,5000000,Хорошее,Удаление
8,9,Ольха серая,1,5.0,4000000,Хорошее,Удаление
9,10,береза,41.5 м.кв.,1.5,50000,Хорошее,Удаление


In [56]:
compress_size('5х14, 4х30, 3х8,2,1')

['5x14', '4x30', '3x8', '2', '1']


'5x14,4x30,3x8,2,1'

In [159]:
from pyautocad.utils import string_to_mtext
from pyautocad import Autocad, ACAD, APoint
import pandas as pd

class _TableStyle:
    class TextTitle:
        style = 'NF_VGP_5.0'
        height = 400.268
    class TextColumnName:
        style = 'NF_VGP_3.0'
        height = 300
    class TextZone:
        style = 'NF_VGP_5.0'
        height = 300
    class TextORM:
        style = 'NF_VGP_3.0'
        height = 300
        h_margin = 100

class TableStyleExists(_TableStyle):
    class TableSize:
        class Column:
            spacings = (
                ('Номер точки', 1005.4436),
                ('Наименование', 6000.1037),
                ('Кол-во,\nшт.', 1427.5616),
                ('Высота,\nм', 1500.0259),
                ('Диаметр\nствола,\nсм', 1572.4902),
                ('Качественное\nсостояние', 4000.0691),
                ('Примечание', 3000.0058)
            )
            full_width = sum([width for name, width in spacings])
        class Row:
            column_name = 1500.0259
            zone = 800.0138
            orm = 800.0138

class TableStyleRemovable(_TableStyle):
    # TODO
    column_titles = [...]
    class TableSize:
        class Column:
            pass
        class Row:
            pass

class Table:
    def __init__(self, style: TableStyleExists | TableStyleRemovable, zone_name: str, data: pd.DataFrame) -> None:
        self._style = style
        self._zone_name = zone_name
        self._data = data
        self._connection_point: tuple[float, float] = (0.0, 0.0)
        self.model = Autocad().model
    
    def _add_line(self, p1: tuple[float, float], p2: tuple[float, float]) -> None:
        return self.model.addLine(APoint(*p1), APoint(*p2))
    
    def _draw_column_titles(self) -> None:
        cell_height = self._style.TableSize.Row.column_name
        full_width = self._style.TableSize.Column.full_width
        self._add_line((0, 0), (full_width, 0))
        self._add_line((0, -cell_height), (full_width, -cell_height))
        x = 0
        self._add_line((0, 0), (0, -cell_height))
        for name, step in self._style.TableSize.Column.spacings:
            x += step
            self._add_line((x, 0), (x, -cell_height))
            text_user_width = step
            insert_point = APoint(x-step, 0)
            text_obj = self.model.AddMText(insert_point, text_user_width, string_to_mtext(name))
            text_obj.Height = TableStyleExists.TextColumnName.height
            text_obj.AttachmentPoint = ACAD.acAttachmentPointMiddleCenter
        self._connection_point = (self._connection_point[0], self._connection_point[1] - cell_height)
    
    def _draw_zone_title(self) -> None:
        cell_height = self._style.TableSize.Row.zone
        full_width = self._style.TableSize.Column.full_width
        y = self._connection_point[1] - cell_height
        self._add_line((0, y), (full_width, y))
        self._add_line(self._connection_point, (0, y))
        self._add_line((full_width, self._connection_point[1]), (full_width, y))
        insert_point = APoint(*self._connection_point)
        text_obj = self.model.AddMText(insert_point, full_width, string_to_mtext(self._zone_name))
        text_obj.Height = TableStyleExists.TextZone.height
        text_obj.AttachmentPoint = ACAD.acAttachmentPointMiddleCenter
        self._connection_point = (self._connection_point[0], self._connection_point[1] - cell_height)
    
    def _draw_orm_row(self, values: list[str]) -> None:
        assert len(values) == len(self._style.TableSize.Column.spacings), "Количество значений строки не равно количеству столбцов таблицы"
        cell_height = self._style.TableSize.Row.orm
        full_width = self._style.TableSize.Column.full_width
        y = self._connection_point[1] - cell_height
        self._add_line((0, y), (full_width, y))
        margin = 100
        x = 0
        i = -1
        self._add_line(self._connection_point, (0, y))
        for name, step in self._style.TableSize.Column.spacings:
            x += step
            self._add_line((x, y), (x, self._connection_point[1]))
            text_user_width = step - margin
            insert_point = APoint(x-step+margin, self._connection_point[1]-margin)
            i += 1
            text_obj = self.model.AddMText(insert_point, text_user_width, string_to_mtext(str(values[i])))
            text_obj.Height = TableStyleExists.TextORM.height
            text_obj.LineSpacingFactor = 0.7
        self._connection_point = (self._connection_point[0], self._connection_point[1] - cell_height)
        
    def draw_table(self) -> None:
        self._draw_column_titles()
        self._draw_zone_title()
        for _, series in self._data.iterrows():
            self._draw_orm_row(series.tolist())

table = Table(TableStyleExists(), 'Zone', df)
table.draw_table()

OSError: [WinError -2147221021] Операция недоступна